In [1]:
import numpy as np
import pandas as pd

import calendar
import datetime as dt
from collections import Counter

In [2]:
deliveries = pd.read_csv(r'Einkaufslieferzeile.csv', on_bad_lines='skip', sep=';', low_memory=False,encoding='utf-8-sig')
deliveries.head()

,Attached to Line No_,Auto Charge Doc_ Line No_,Auto Charge Doc_ No_,Billing Entry No_,Billing Reference No_,Blanket Order Line No_,Blanket Order No_,Budgeted FA No_,Buy-from Vendor No_,Contract Line No_,...,Use Duplication List,Use Tax,VAT Base Amount,VAT Bus_ Posting Group,VAT Calculation Type,VAT Prod_ Posting Group,VAT _,VDP Code,Variant Code,timestamp
0,0,0,NaN,0,NaN,0,NaN,NaN,K00421,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,000000005295C63A
1,0,0,NaN,0,NaN,0,NaN,NaN,K02028,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052EC85AD
2,0,0,NaN,0,NaN,0,NaN,NaN,K04610,0,...,0,0,"0,00000000000000000000",EU,1,MWST19,"0,00000000000000000000",NaN,NaN,0000000052B4AE3C
3,0,0,NaN,0,NaN,0,NaN,NaN,K04607,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4D12C
4,0,0,NaN,0,NaN,10000,RB19/0025,NaN,K04137,0,...,0,0,"0,00000000000000000000",IL,0,MWST19,"19,00000000000000000000",NaN,NaN,0000000052B4B8F5


In [3]:
deliveries.shape

(6012, 166)

In [4]:
# drop columns where all NaN values
deliveries=deliveries.dropna(axis=1, how='all')
deliveries.shape

(6012, 129)

In [5]:
# drop columns where all 0 values
deliveries=deliveries.loc[:, (deliveries != 0).any(axis=0)]
deliveries.shape

(6012, 103)

In [6]:
# drop columns where over 50% of values are null values
deliveries=deliveries.loc[:, deliveries.isnull().mean()<0.5]
deliveries.shape

(6012, 90)

In [7]:
# get list with column names
list(deliveries.columns)

['Blanket Order Line No_',
 'Buy-from Vendor No_',
 'Cross-Reference Type No_',
 'Demand Query Line No_',
 'Document No_',
 'Item Rcpt_ Entry No_',
 'Line No_',
 'No_',
 'Order Line No_',
 'Order No_',
 'Pay-to Vendor No_',
 'Pos_ No_',
 'Prod_ Order Line No_',
 'Routing Reference No_',
 'Allow Invoice Disc_',
 'Balance',
 'Bin Code',
 'Buy-from Address',
 'Buy-from City',
 'Buy-from Contact',
 'Buy-from Post Code',
 'Buy-from Vendor Name 2',
 'Buy-from Vendor Name',
 'Contract Amount',
 'Correction',
 'Cross-Reference Type',
 'Description 2',
 'Description',
 'Dimension Set ID',
 'Direct Unit Cost Price Factor',
 'Direct Unit Cost',
 'Expected Receipt Date',
 'Expected Receipt Time',
 'FA Posting Date',
 'Gen_ Bus_ Posting Group',
 'Gen_ Prod_ Posting Group',
 'Gross Weight',
 'Indirect Cost _',
 'Item Category Code',
 'Item Charge Base Amount',
 'Job Currency Factor',
 'Job Line Amount (LCY)',
 'Job Line Amount',
 'Job Line Disc_ Amount (LCY)',
 'Job Line Discount Amount',
 'Job Line

In [8]:
# drop columns that don't seem relevant to our challenge
to_drop=['Cross-Reference Type No_', 'Demand Query Line No_', 'Document No_', 'Item Rcpt_ Entry No_',
 'Line No_', 'Order Line No_', 'Pay-to Vendor No_', 'Pos_ No_', 'Prod_ Order Line No_', 'Routing Reference No_', 'Allow Invoice Disc_', 'Balance',
 'Bin Code', 'Contract Amount', 'Correction', 'Cross-Reference Type', 'Description 2', 'Dimension Set ID', 'Direct Unit Cost Price Factor',
 'Direct Unit Cost', 'Gen_ Bus_ Posting Group', 'Gen_ Prod_ Posting Group', 'Gross Weight', 'Indirect Cost _', 'Item Charge Base Amount',
 'Job Currency Factor', 'Job Line Amount (LCY)', 'Job Line Amount', 'Job Line Disc_ Amount (LCY)', 'Job Line Discount Amount', 'Job Line Discount _',
 'Job Total Price (LCY)', 'Job Unit Price (LCY)', 'Job Unit Price', 'Line Discount _', 'Net Weight', 'Overhead Rate', 'Pack Sample Quantity', 'Price Factor', 
 'Qty_ per Unit of Measure', 'Salvage Value', 'Shortcut Dimension 1 Code', 'Shortcut Dimension 2 Code','Qty_ Invoiced (Base)','Quantity (Base)',
 'Type', 'Unit Cost (LCY)', 'Unit Cost', 'Unit Price (LCY)', 'Unit Volume', 'Unit of Measure (Cross Ref_)', 'Unit of Measure Code', 'Location Code',
 'Unit of Measure', 'Units per Parcel', 'VAT Base Amount', 'VAT Bus_ Posting Group', 'VAT Calculation Type', 'VAT Prod_ Posting Group','Blanket Order Line No_',
 'VAT _', 'Buy-from Vendor Name 2','Buy-from Address', 'timestamp','Item Category Code','Posting Group','Description','Buy-from Contact','Job Total Price','Quantity',
  'Order Quantity', 'Qty_ Rcd_ Not Invoiced', 'Quantity Invoiced']
deliveries.drop(to_drop, inplace=True, axis=1)

In [9]:
# there are many columns with dates, let's analyse them with the goal of dropping the ones that are not needed
dates=deliveries[[
 'Expected Receipt Date', 'Expected Receipt Time', 'FA Posting Date', 'Lead Time Calculation',  'Planned Receipt Date',
 'Order Date', 'Original Date', 'Posting Date', 'Promised Receipt Date', 'Really Receipt DateTime', 'Requested Receipt Date']]
dates.head(3)

,Expected Receipt Date,Expected Receipt Time,FA Posting Date,Lead Time Calculation,Planned Receipt Date,Order Date,Original Date,Posting Date,Promised Receipt Date,Really Receipt DateTime,Requested Receipt Date
0,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,23.09.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,16.12.2019 23:00:00,17.12.2019 00:00:00
1,17.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,17.12.2019 00:00:00,15.11.2019 00:00:00,17.12.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,01.01.1753 00:00:00,17.12.2019 00:00:00
2,19.12.2019 00:00:00,01.01.1753 00:00:00,01.01.1753 00:00:00,NaN,19.12.2019 00:00:00,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,01.01.1753 00:00:00,19.12.2019 00:00:00


**Date columns to keep (definitions)**

Order Date: date the order was created. 

Posting Date: date you want as the Posting Date on the Ledger Entry tables when the order is Shipped and Invoiced.

Promised Receipt Date: date your vendor promised they would deliver the goods. NAV does not change this date, so it allows you to set a marker to measure your vendor’s performance against what was promised.

Requested Receipt Date: date you wanted to receive the goods from the vendor. This date is not recalculated in NAV, so once it has been established, it gives you a marker to measure your vendor’s performance against the requested date.

Really Receipt DateTime

In [10]:
to_drop=['Expected Receipt Time','FA Posting Date','Lead Time Calculation','Planned Receipt Date','Original Date',]
deliveries.drop(to_drop, inplace=True, axis=1)
deliveries.head()

,Buy-from Vendor No_,No_,Order No_,Buy-from City,Buy-from Post Code,Buy-from Vendor Name,Expected Receipt Date,Order Date,Posting Date,Promised Receipt Date,Really Receipt DateTime,Requested Receipt Date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019 00:00:00,23.09.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,16.12.2019 23:00:00,17.12.2019 00:00:00
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019 00:00:00,15.11.2019 00:00:00,30.12.2019 00:00:00,17.12.2019 00:00:00,01.01.1753 00:00:00,17.12.2019 00:00:00
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019 00:00:00,19.12.2019 00:00:00,30.12.2019 00:00:00,19.12.2019 00:00:00,01.01.1753 00:00:00,19.12.2019 00:00:00
3,K04607,KS00525776-1,EB19/3737,Ebersbach,73061,Bodycote,06.01.2020 00:00:00,03.12.2019 00:00:00,30.12.2019 00:00:00,06.01.2020 00:00:00,01.01.1753 00:00:00,18.12.2019 00:00:00
4,K04137,MV00491619,EB19/2532,Velbert,42551,Heismann Drehtechnik,18.12.2019 00:00:00,03.07.2019 00:00:00,30.12.2019 00:00:00,18.12.2019 00:00:00,01.01.1753 00:00:00,18.12.2019 00:00:00


In [11]:
# rename columns
deliveries.columns = deliveries.columns.str.replace(" ", "_").str.lower()
deliveries=deliveries.rename(columns={'buy-from_vendor_no_': 'vendor_no',"no_": "delivery_no","order_no_": "order_no",'buy-from_city':'city',
'buy-from_post_code':'postcode','buy-from_vendor_name':'vendor_name','expected_receipt_date':'expected_date','promised_receipt_date': 'promised_date', 
'really_receipt_datetime':'delivery_date', 'requested_receipt_date':'requested_date'})
deliveries.vendor_name=deliveries.vendor_name.str.replace(",", " ") # commas would create additional columns when exporting to csv
list(deliveries.columns)

['vendor_no',
 'delivery_no',
 'order_no',
 'city',
 'postcode',
 'vendor_name',
 'expected_date',
 'order_date',
 'posting_date',
 'promised_date',
 'delivery_date',
 'requested_date']

In [12]:
# drop unnecessary characters
deliveries[['expected_date','order_date','posting_date','promised_date','delivery_date','requested_date']] = deliveries[['expected_date','order_date','posting_date','promised_date','delivery_date','requested_date']].applymap(lambda x: str(x)[:-9])

deliveries.head(3)

,vendor_no,delivery_no,order_no,city,postcode,vendor_name,expected_date,order_date,posting_date,promised_date,delivery_date,requested_date
0,K00421,00451185,EB19/2893,Altena-Dahle,58754,Möhling GmbH & Co. KG,17.12.2019,23.09.2019,30.12.2019,17.12.2019,16.12.2019,17.12.2019
1,K02028,17000040,EB19/3532,Mörfelden-Walldorf,64546,RS Components GmbH,17.12.2019,15.11.2019,30.12.2019,17.12.2019,01.01.1753,17.12.2019
2,K04610,KS00520714,EB19/3570,Carei Jud. Satu Mare,RO - 445100,Taygan Metal Press SRL,19.12.2019,19.12.2019,30.12.2019,19.12.2019,01.01.1753,19.12.2019


In [13]:
# Dates with a value of 01.01.1753 and 09.09.2099 mean that the field is blank or there's been an error in the input. There are also dates with the year 2033.
# We will filter out these rows
deliveries=deliveries[deliveries.promised_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.delivery_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.expected_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.order_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.posting_date.str.contains("1753|2033|2099")==False]
deliveries=deliveries[deliveries.requested_date.str.contains("1753|2033|2099")==False]
deliveries.shape

(4152, 12)

In [14]:
# typecast
deliveries = deliveries.astype({'order_date': 'datetime64','posting_date': 'datetime64','promised_date': 'datetime64',
'delivery_date': 'datetime64','requested_date': 'datetime64','expected_date':'datetime64'})
deliveries.dtypes

vendor_no                 object
delivery_no               object
order_no                  object
city                      object
postcode                  object
vendor_name               object
expected_date     datetime64[ns]
order_date        datetime64[ns]
posting_date      datetime64[ns]
promised_date     datetime64[ns]
delivery_date     datetime64[ns]
requested_date    datetime64[ns]
dtype: object

In [15]:
# We will drop rows that have a delivery, promised or order date later than 01.03.2022
# Also, rows where order date is later than delivery, expected, requested or posting dates will be dropped. We assume this is an input error.
deliveries = deliveries[(deliveries['promised_date']< '2022-03-01')&(deliveries['delivery_date']< '2022-03-01')&(deliveries['order_date']< '2022-03-01')]
deliveries = deliveries[(deliveries['promised_date']> deliveries['order_date'])&(deliveries['delivery_date']> deliveries['order_date'])&(deliveries['expected_date']> deliveries['order_date'])&(deliveries['requested_date']> deliveries['order_date'])&(deliveries['posting_date']> deliveries['order_date'])]

deliveries.shape

(2334, 12)

In [16]:
# drop rows with nan values
deliveries.isna().sum()
deliveries = deliveries.dropna()
deliveries.shape

# drop duplicates
deliveries.drop_duplicates(keep='first')
deliveries.shape

(2332, 12)

Our analysis is only focused in Germany, hence, we will drop all rows not corresponding to this country. Since the only location data included in our dataframe is city and postcode, longitude and latitude columns will also be included.

In [17]:
# uploading file with additional location data from Germany
# Source: https://github.com/zauberware/postal-codes-json-xml-csv/blob/master/data/DE.zip
de_states = pd.read_csv(r'zipcodes.de.csv', on_bad_lines='skip',encoding='unicode_escape')
de_states = de_states[['zipcode', 'place', 'state', 'community', 'latitude', 'longitude']]
de_states['country']='Germany'
de_states=de_states.rename(columns={'zipcode': 'postcode',"place": "city"})
de_states = de_states.astype({'postcode': 'str'})
de_states.head()
de_states.dtypes

postcode      object
city          object
state         object
community     object
latitude     float64
longitude    float64
country       object
dtype: object

In [18]:
# we merge deliveries and de_states dataframes on postcode
# drop duplicated column city_y
germany=pd.merge(deliveries,de_states, on="postcode", how="left").drop(columns="city_y").rename(columns={"city_x": "city"})
germany.head()
print(Counter(germany.country))

Counter({'Germany': 2618, nan: 284})


In [19]:
# As we can see, most rows correspond to Germany. We will drop all rows that do not.
germany=germany.dropna(subset=["country"])
germany.shape
germany.head()

# reorganize columns 
germany = germany[['vendor_no', 'vendor_name', 'order_no','delivery_no', 'city', 'postcode','state','community','country','longitude', 'latitude','expected_date', 
'order_date', 'posting_date', 'promised_date', 'requested_date', 'delivery_date']]
germany.head()

,vendor_no,vendor_name,order_no,delivery_no,city,postcode,state,community,country,longitude,latitude,expected_date,order_date,posting_date,promised_date,requested_date,delivery_date
1,K00016,Höttges,EB19/3307,40063527,Berlin,13437,Berlin,"Berlin, Stadt",Germany,13.3284,52.5905,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-18
2,K00016,Höttges,EB19/3307,40063526,Berlin,13437,Berlin,"Berlin, Stadt",Germany,13.3284,52.5905,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-16
3,K00268,Electrovac Hacht & Huber GmbH,EB19/2763,27602381,Salzweg,94121,Bayern,Landkreis Passau,Germany,13.4833,48.6167,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-13,2019-12-19
6,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Brandenburg,Landkreis Oberhavel,Germany,13.3876,52.7300,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01
7,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Brandenburg,Landkreis Oberhavel,Germany,13.2746,52.8261,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01


In [20]:
# we also want to add a column that will lt us see how the difference between the promised day of delivery and the actual delivery date
germany['days_variance'] = (germany.promised_date - germany.delivery_date).dt.days
germany.days_variance.dtypes

dtype('int64')

In [21]:
germany.head()

,vendor_no,vendor_name,order_no,delivery_no,city,postcode,state,community,country,longitude,latitude,expected_date,order_date,posting_date,promised_date,requested_date,delivery_date,days_variance
1,K00016,Höttges,EB19/3307,40063527,Berlin,13437,Berlin,"Berlin, Stadt",Germany,13.3284,52.5905,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-18,-5
2,K00016,Höttges,EB19/3307,40063526,Berlin,13437,Berlin,"Berlin, Stadt",Germany,13.3284,52.5905,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-16,-3
3,K00268,Electrovac Hacht & Huber GmbH,EB19/2763,27602381,Salzweg,94121,Bayern,Landkreis Passau,Germany,13.4833,48.6167,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-13,2019-12-19,-6
6,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Brandenburg,Landkreis Oberhavel,Germany,13.3876,52.7300,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01,-225
7,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Brandenburg,Landkreis Oberhavel,Germany,13.2746,52.8261,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01,-225


In [22]:
# after calculating the days variance, we want to establish if the delivery was on time or not
germany["otd_ld"]=germany["days_variance"].apply(lambda value: 'On-Time Delivery' if value>= 0 
else 'Late Delivery')
germany

,vendor_no,vendor_name,order_no,delivery_no,city,postcode,state,community,country,longitude,latitude,expected_date,order_date,posting_date,promised_date,requested_date,delivery_date,days_variance,otd_ld
1,K00016,Höttges,EB19/3307,40063527,Berlin,13437,Berlin,"Berlin, Stadt",Germany,13.3284,52.5905,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-18,-5,Late Delivery
2,K00016,Höttges,EB19/3307,40063526,Berlin,13437,Berlin,"Berlin, Stadt",Germany,13.3284,52.5905,2019-12-13,2019-10-29,2019-12-30,2019-12-13,2019-12-13,2019-12-16,-3,Late Delivery
3,K00268,Electrovac Hacht & Huber GmbH,EB19/2763,27602381,Salzweg,94121,Bayern,Landkreis Passau,Germany,13.4833,48.6167,2019-12-13,2019-08-22,2019-12-30,2019-12-13,2019-12-13,2019-12-19,-6,Late Delivery
6,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Brandenburg,Landkreis Oberhavel,Germany,13.3876,52.7300,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01,-225,Late Delivery
7,K03806,SURFACED GmbH,EB19/3814,KP0064C004,Oranienburg,16515,Brandenburg,Landkreis Oberhavel,Germany,13.2746,52.8261,2019-12-20,2019-11-12,2020-09-01,2019-12-20,2019-12-20,2020-08-01,-225,Late Delivery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,K04152,Klumpp GmbH & Co. KG,EB21/3677,HDE0H03449,Bretten,75015,Baden-Württemberg,Karlsruhe,Germany,8.7075,49.0369,2022-01-28,2021-12-16,2022-01-31,2022-01-28,2022-01-28,2022-01-30,-2,Late Delivery
2897,K00910,Adelhelm LubriCoat GmbH,EB21/3506,17200137,Eningen,72800,Baden-Württemberg,Reutlingen,Germany,9.2595,48.4869,2022-01-02,2021-11-30,2022-01-02,2022-01-02,2022-01-28,2022-01-31,-29,Late Delivery
2898,K04659,Robert Bosch GmbH,EB22/0023,HDE0001158,Bamberg,96052,Bayern,Bamberg,Germany,10.8951,49.9083,2022-01-28,2021-12-17,2022-01-02,2022-01-28,2022-01-28,2022-01-31,-3,Late Delivery
2899,K04128,Albert Weber GmbH,EB21/3732,HD00100047,Neuenbürg,75305,Baden-Württemberg,Enzkreis,Germany,8.5957,48.8452,2022-01-28,2021-04-15,2022-01-31,2022-01-28,2021-05-08,2022-01-02,26,On-Time Delivery


In [23]:
# export as csv file for MySQL
germany.to_csv(r'de_deliveries.csv', index=False)

# export to Excel file for Tableau
germany.to_excel (r'de_deliveries.xlsx', index = None, header=True)